In [22]:
import os
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'
print(jax.devices())

[CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3), CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7)]


In [11]:
import jax
import jax.numpy as jnp
from jax import shard_map
from jax.sharding import Mesh, PartitionSpec as PS, NamedSharding
from typing import Optional
from functools import partial

In [5]:
mesh = jax.make_mesh((4, 2), ('x', 'y'))

In [15]:
a = jnp.arange(8*16).reshape(8,16)
b = jnp.arange(16*4).reshape(16,4)


In [21]:
jax.debug.visualize_array_sharding(a)
jax.debug.visualize_array_sharding(b)

                                                  
                                                  
                                                  
                                                  
                                                  
                      CPU 0                       
                                                  
                                                  
                                                  
                                                  
                                                  

         
         
         
         
         
  CPU 0  
         
         
         
         
         

In [27]:
@partial(shard_map,mesh=mesh,in_specs=(PS('x','y'),PS('y',None)),out_specs=PS('x',None))
def matmul_basic(a_block, b_block):
    c_partialsum = jnp.dot(a_block, b_block)
    print(c_partialsum)
    c_block = jax.lax.psum(c_partialsum, 'y')
    print(c_block)
    return c_block


In [28]:
c = matmul_basic(a,b)

On TFRT_CPU_0 at mesh coordinates (x, y,) = (0, 0):
[[ 560  588  616  644]
 [2352 2508 2664 2820]]

On TFRT_CPU_1 at mesh coordinates (x, y,) = (0, 1):
[[ 4400  4492  4584  4676]
 [10288 10508 10728 10948]]

On TFRT_CPU_2 at mesh coordinates (x, y,) = (1, 0):
[[4144 4428 4712 4996]
 [5936 6348 6760 7172]]

On TFRT_CPU_3 at mesh coordinates (x, y,) = (1, 1):
[[16176 16524 16872 17220]
 [22064 22540 23016 23492]]

On TFRT_CPU_4 at mesh coordinates (x, y,) = (2, 0):
[[ 7728  8268  8808  9348]
 [ 9520 10188 10856 11524]]

On TFRT_CPU_5 at mesh coordinates (x, y,) = (2, 1):
[[27952 28556 29160 29764]
 [33840 34572 35304 36036]]

On TFRT_CPU_6 at mesh coordinates (x, y,) = (3, 0):
[[11312 12108 12904 13700]
 [13104 14028 14952 15876]]

On TFRT_CPU_7 at mesh coordinates (x, y,) = (3, 1):
[[39728 40588 41448 42308]
 [45616 46604 47592 48580]]

On TFRT_CPU_0 at mesh coordinates (x, y,) = (0, 0):
[[ 4960  5080  5200  5320]
 [12640 13016 13392 13768]]

On TFRT_CPU_1 at mesh coordinates (x, y,) = 

In [18]:
c.shape

(8, 4)